Archivo para generar los archivos para poder realizar los submission de forma manual en Kaggle

Durante el experimento me encontre con la dificultad de que mis notebooks corrian hasta el primer archivo para realizar el submission en kaggle y, luego, no podía avanzar. Para poder subasanar este issue, logre realizar esta query para poder generar los archivos y realizar los submission de forma manual.

In [ ]:
# ===========================================
# Generación de archivos de envío para Kaggle
# ===========================================

# Cargo librerías necesarias
if(!require("data.table")) install.packages("data.table")
library(data.table)

# Parámetros del experimento
PARAM <- list()
PARAM$experimento <- "WF_195012"
PARAM$fts_semillas <- 1:20  # ejemplo: 5 semillas
PARAM$kaggle <- list(
  competencia = "labio-i-2025-ba-analista-sr",
  cortes = seq(10000, 12000, by = 500)
)

# 1️⃣ Leo el archivo local de predicciones
#Descargo el archivo de predicciones generado por el experimento
#Cambiar ruta al archivo de predicciones descargado previamente
# Asegurate de que 'prediccion.txt' tenga columnas: numero_de_cliente, prob
tb_prediccion <- fread("C:\\Users\\angel\\Downloads\\exp_WF_195012_prediccion.txt", sep = "\t", header = TRUE)

# 2️⃣ Ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

# 3️⃣ Creo carpeta de salida si no existe
if(!dir.exists("kaggle")) dir.create("kaggle")

# 4️⃣ Genero archivos con distintos puntos de corte
for (envios in PARAM$kaggle$cortes) {
  
  # Reinicio columna Predicted
  tb_prediccion[, Predicted := 0L]
  
  # Marco con 1 los primeros 'envios' registros (mayor probabilidad)
  tb_prediccion[1:envios, Predicted := 1L]
  
  # Defino el nombre del archivo de salida
  archivo_kaggle <- paste0(
    "./kaggle/KA_",
    PARAM$experimento, "_",
    "s", length(PARAM$fts_semillas), "_",
    envios, ".csv"
  )
  
  # 5️⃣ Grabo el archivo CSV con las columnas requeridas
  fwrite(
    tb_prediccion[, .(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )
  
  cat("Archivo generado:", archivo_kaggle, "\n")
}

cat("\n✅ Archivos generados correctamente en la carpeta 'kaggle/'\n")
